# Find Multiple Mutations in Cis

For a user specified gene, this notebook finds if there are multiple types of mutations 

In [1]:
import cptac
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plot_utils as p
import warnings
warnings.filterwarnings('ignore')

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#import CPTAC data
en = cptac.Endometrial()
hn = cptac.Hnscc()
l = cptac.Luad()
o = cptac.Ovarian()
c = cptac.Ccrcc()
col = cptac.Colon()
g = cptac.Gbm()
b = cptac.Brca()
ls = cptac.Lscc()

KeyboardInterrupt: 

In [ ]:
def figure1_plot_mutations(cis_gene, dflist = None, names_of_df=None, title=None, save_to_path=None):
    number_of_df = len(dflist)

    allLabels = []
    for df in dflist:
        #get the labels for each and make a combined label that they'll all use
        mutation = df["All_Mutations_"+cis_gene]
        labels = list(set(mutation))

        allLabels.append(labels)

    flat_list = [item for sublist in allLabels for item in sublist]
    all_labels = list(set(flat_list))
    all_labels.sort()
    allLabels = all_labels

#     For each df, add na to their labels if it doesn't exist in all_labels
    labels_for_each_df = []
    frequencies_for_each_df = []
    for df in dflist:
        mutation = df["All_Mutations_"+cis_gene].tolist()
        mutationlist = list(set(mutation))
        mutationlist.sort()
        ordered_mut_list = []
        match = True
        mutPosition = 0

        for position in range(len(all_labels)):
            try:

                if mutationlist[mutPosition] == all_labels[position]:
                    ordered_mut_list.append(mutationlist[mutPosition])
                    mutPosition += 1

                else:
                    ordered_mut_list.append("na")

            except IndexError:
                ordered_mut_list.append("na")


        labels_for_each_df.append(ordered_mut_list)

        #get the freq of each mutation type
        freq = []
        for mutation_type in ordered_mut_list:
            freq.append(mutation.count(mutation_type))

        PercentFreq = [x*100 / sum(freq) for x in freq]
        frequencies_for_each_df.append(PercentFreq)


    #Now plot it using arrays
    width = 0.1
    x = np.arange(len(allLabels))
    a4_dims = (140, 80) #dimensions for bigger plot
    fig, ax = plt.subplots(figsize=a4_dims)
    for position in range(0, number_of_df):
        r = ax.bar(x+(width*position), frequencies_for_each_df[position], width,label=names_of_df[position], alpha=.5, linewidth=0)



    ax.set_ylabel('Percent Sample', size = 120)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(allLabels)
    ax.legend()
    ax.set_xticklabels(allLabels, size = 120)
    ax.set_ylim(0,100)
    ax.legend(prop={'size': 120})
    ax.yaxis.set_tick_params(labelsize=120)


    plt.setp(ax.get_xticklabels(),rotation= 90)
    plt.tight_layout()
    if save_to_path == None:
        plt.savefig("step_1.png",dpi = 300)
    else:
        plt.savefig(save_to_path)

    plt.show()

In [ ]:
def find_multi_cis_mut(cancer_obj,cis_gene):
    #Get all somatic mutations 
    mutations = cancer_obj.get_somatic_mutation()
    #get just somatic mutations for cis_gene
    cis_mut = mutations[mutations.Gene == cis_gene]
    cis_mut = cis_mut[["Mutation"]]
    
    #simplfy names 
    cis_mut["Mutation"].where(cis_mut["Mutation"] != 'Missense_Mutation', 'Missense', inplace = True) # replaces when false
    #cis_mut["Mutation"].where(cis_mut["Mutation"] != 'Wildtype_Tumor', 'Wildtype', inplace = True)
    #cis_mut["Mutation"].where(cis_mut["Mutation"] != 'No_Mutation', 'Wildtype', inplace = True)
    cis_mut["Mutation"].where(cis_mut["Mutation"] != 'In_Frame_Del', 'Indel', inplace = True)
    cis_mut["Mutation"].where(cis_mut["Mutation"] != 'In_Frame_Ins', 'Indel', inplace = True)
    
    cis_mut["Mutation"].where(cis_mut["Mutation"] != 'nonframeshift insertion', 'Indel', inplace = True)
    cis_mut["Mutation"].where(cis_mut["Mutation"] != 'frameshift deletion', 'Truncation', inplace = True)
    cis_mut["Mutation"].where(cis_mut["Mutation"] != 'nonsynonymous SNV', 'Missense', inplace = True)
    
    #group Truncation
    cis_mut["Mutation"].where(cis_mut["Mutation"] != 'Nonsense_Mutation', 'Truncation', inplace = True)
    cis_mut["Mutation"].where(cis_mut['Mutation'] != 'Frame_Shift_Del', 'Truncation', inplace = True)
    cis_mut["Mutation"].where(cis_mut['Mutation'] != 'Frame_Shift_Ins', 'Truncation', inplace = True)
    
    
    # Multiple somatic mutations are indicated using multiple rows with the same patient ids.
    # We will group the rows by patient ids and then relabel the columns that have multiple mutations 
    
    cis_mut = cis_mut.groupby(level=['Patient_ID'], sort=False).agg( ','.join)
    cis_mut = cis_mut.replace(regex=[',' + cis_gene], value='')
    cis_mut.Mutation = cis_mut.Mutation.apply(lambda x: 'Multiple_Missense' if 'Missense,Missense' in x else x)
    cis_mut.Mutation = cis_mut.Mutation.apply(lambda x: 'Multiple_Frame_Shift_Del' if 'Frame_Shift_Del,Frame_Shift_Del' in x else x)
    cis_mut.Mutation = cis_mut.Mutation.apply(lambda x: 'Multiple_Splice_Site' if 'Splice_Site,Splice_Site' in x else x)
   
   #get CNV mutations 
    mut_copy_num = cancer_obj.get_genotype_all_vars(cis_gene, mutations_filter = ["Deletion","Amplification","Wildtype_Tumor"])
    mut_copy_num = mut_copy_num[mut_copy_num['Mutation'].isin(["Deletion","Amplification","Wildtype_Tumor"])]
    mut_copy_num = mut_copy_num.rename(columns={'Mutation': 'CNV_Mutation'})
  
    mut_copy_num = mut_copy_num["CNV_Mutation"]
    mutations = pd.merge(mut_copy_num, cis_mut, on="Patient_ID", how = "outer")
   
     
    # If mutation column is NaN replace with value in Mutation_Copy_Num
    mutations['Mutation'] = mutations.apply(
        lambda row: row['CNV_Mutation'] if pd.isnull(row['Mutation']) else row['Mutation'], axis=1)
   # mutations = mutations.CNV_Mutation.fillna('', inplace=True)
    #Make column All_Mutation which is a list of all mutations 
    mutations = mutations.replace(np.nan, '', regex=True)
    mutations['All_Mutations'] = mutations.apply(
        lambda row: row["CNV_Mutation"] + ", " + row["Mutation"] if (row["CNV_Mutation"] != row["Mutation"] and row["CNV_Mutation"] != "") else row['Mutation'], axis=1)
    mutations = mutations.rename(columns={'All_Mutations': 'All_Mutations_' + cis_gene})
    return mutations


In [ ]:
#Call function to create dataframe with all mutations 
cis_gene = "PTEN"

endo = find_multi_cis_mut(en, cis_gene)
gbm = find_multi_cis_mut(g, cis_gene)
hnscc = find_multi_cis_mut(hn, cis_gene)
ovar = find_multi_cis_mut(o, cis_gene)
cc = find_multi_cis_mut(c, cis_gene)
colon = find_multi_cis_mut(col, cis_gene)
ld = find_multi_cis_mut(l, cis_gene)
brca = find_multi_cis_mut(b, cis_gene)
lscc = find_multi_cis_mut(ls, cis_gene)


In [ ]:
dfs = [endo, gbm, hnscc, ovar, cc, colon, ld, brca, lscc]
names = ['Endometrial', 'Glioblastoma', 'Head and Neck', 'Ovarian', 'Renal', "Colon" ,'Lung Adenocarcinoma', 'Breast',"Lung Squamous"]
figure1_plot_mutations("PTEN",dfs, names)

In [73]:
#Call function to create dataframe with all mutations 
cis_gene = "EGFR"

endo = find_multi_cis_mut(en, cis_gene)
gbm = find_multi_cis_mut(g, cis_gene)
hnscc = find_multi_cis_mut(hn, cis_gene)
ovar = find_multi_cis_mut(o, cis_gene)
cc = find_multi_cis_mut(c, cis_gene)
colon = find_multi_cis_mut(col, cis_gene)
ld = find_multi_cis_mut(l, cis_gene)
brca = find_multi_cis_mut(b, cis_gene)
lscc = find_multi_cis_mut(ls, cis_gene)


In [74]:

dfs = [endo, gbm, hnscc, ovar, cc, colon, ld, brca, lscc]
names = ['Endometrial', 'Glioblastoma', 'Head and Neck', 'Ovarian', 'Renal', "Colon" ,'Lung Adenocarcinoma', 'Breast',"Lung Squamous"]
figure1_plot_mutations("EGFR",dfs, names)
